In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import joblib
import os

In [2]:
# Training Stock (Various Stocks for various data predictions)
TICKERS = ["RELIANCE.NS", "TCS.NS", "HDFCBANK.NS", "TATASTEEL.NS", "SUNPHARMA.NS", "PAYTM.NS", "IDEA.NS"]
LOOKBACK = 60
EPOCHS = 40
BATCH_SIZE = 16

In [3]:
# Function to calculate Relative Strength Index (RSI)
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    
    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [4]:
# dataset Creation
def create_dataset(dataset, look_back=60):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        X.append(dataset[i:(i + look_back)])
        
        if dataset[i + look_back, 0] > 0: 
            Y.append(1)
        else:
            Y.append(0)
            
    return np.array(X), np.array(Y)

In [5]:
# Hybrid Model Training (Returns + RSI)
def train_general_model():
    print(f"🚀 Starting Hybrid Training (Returns + RSI)...")
    
    all_X = []
    all_y = []
    
    scaler = MinMaxScaler(feature_range=(-1, 1))

    for ticker in TICKERS:
        print(f"📥 Processing {ticker}...")
        df = yf.download(ticker, period="10y", progress=False)
        
        if df.empty or len(df) < 200:
            continue

        df['Return'] = df['Close'].pct_change()
        df['RSI'] = calculate_rsi(df['Close'])
        
        df.dropna(inplace=True)
        
        data = df[['Return', 'RSI']].values
        
        scaled_data = scaler.fit_transform(data)
        
        x_stock, y_stock = create_dataset(scaled_data, LOOKBACK)
        
        all_X.append(x_stock)
        all_y.append(y_stock)

    X_train = np.concatenate(all_X)
    y_train = np.concatenate(all_y)
    
    print(f"🧠 Training on {len(X_train)} samples...")

    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(LOOKBACK, 2)),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True)),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences=False)),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Dense(32, activation='relu'),
        
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)
    
    save_dir = "app/models"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    model.save(f"{save_dir}/lstm_model.h5")
    joblib.dump(scaler, f"{save_dir}/scaler.gz")
    
    print("\n✅ Robust Hybrid Model Saved!")

In [6]:
if __name__ == "__main__":
    train_general_model()

🚀 Starting Hybrid Training (Returns + RSI)...
📥 Processing RELIANCE.NS...


/var/folders/d1/rd_d53zj0sz84gg876zdlt5m0000gn/T/ipykernel_47781/3603605256.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="10y", progress=False)


📥 Processing TCS.NS...


/var/folders/d1/rd_d53zj0sz84gg876zdlt5m0000gn/T/ipykernel_47781/3603605256.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="10y", progress=False)


📥 Processing HDFCBANK.NS...


...

2025-11-28 10:44:54.818651: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-11-28 10:44:54.818676: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-11-28 10:44:54.818689: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-11-28 10:44:54.818894: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-28 10:44:54.819252: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/40


2025-11-28 10:44:56.567639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-11-28 10:44:56.678413: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


957/957 [==============================] - 37s 37ms/step - loss: 0.6262 - accuracy: 0.6661
Epoch 2/40
957/957 [==============================] - 36s 37ms/step - loss: 0.6200 - accuracy: 0.6716
Epoch 3/40
957/957 [==============================] - 35s 36ms/step - loss: 0.6216 - accuracy: 0.6699
Epoch 4/40
957/957 [==============================] - 34s 35ms/step - loss: 0.6541 - accuracy: 0.6272
Epoch 5/40
957/957 [==============================] - 34s 35ms/step - loss: 0.6570 - accuracy: 0.6153
Epoch 6/40
957/957 [==============================] - 35s 37ms/step - loss: 0.6541 - accuracy: 0.6268
Epoch 7/40
957/957 [==============================] - 35s 36ms/step - loss: 0.6265 - accuracy: 0.6622
Epoch 8/40
957/957 [==============================] - 34s 36ms/step - loss: 0.6195 - accuracy: 0.6683
Epoch 9/40
957/957 [==============================] - 35s 37ms/step - loss: 0.6208 - accuracy: 0.6697
Epoch 10/40
957/957 [==============================] - 36s 37ms/step - loss: 0.6164 - accurac

/Users/vedantkanoje/Desktop/Trade-sentry/Backend/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
